In [1]:
# Standard libraries: https://docs.python.org/3/library/
import math

# Analysis and plotting modules
import pandas as pd
# import plotly

# cadCAD configuration modules
from cadCAD.configuration.utils import config_sim
from cadCAD.configuration import Experiment

# cadCAD simulation engine modules
from cadCAD.engine import ExecutionMode, ExecutionContext
from cadCAD.engine import Executor

import pandas as pd
import numpy as np
from random import normalvariate
import plotly.express as px

from cadCAD.configuration.utils import config_sim
from cadCAD.configuration import Experiment
from cadCAD.engine import ExecutionContext, Executor

In [2]:
initial_state = {
    'total_population': 80000,
    'potential_adopters': 2400,
}

In [3]:
system_params = {
    'fraction_adopt': [0.6],
    'advertising_adopt': [0.04],
    'word_ofmouth': [0.16], 
}


In [4]:
def p_willing_popu(params, 
                    substep, 
                    state_history, 
                    previous_state):
    

    
    popu_willing_adopt = params['fraction_adopt'] * previous_state['total_population']
    
    # Output
    return {'delta_population': - popu_willing_adopt}


In [5]:
def p_popu_increase(params, 
                    substep, 
                    state_history, 
                    previous_state):
    

    
    popu_willing_adopt = 0.00058  * previous_state['total_population']
    
    # Output
    return {'delta_population': popu_willing_adopt}

In [6]:
def p_popu_adopt(params, 
                    substep, 
                    state_history, 
                    previous_state):
    
    popu_adopt = params['fraction_adopt'] * previous_state['total_population']
    
    
    
    # Output
    
    return {'delta_potential_adopters': popu_adopt}

In [7]:
def p_app_downloads(params, 
                    substep, 
                    state_history, 
                    previous_state):
    
    popu_adopt = (params['word_ofmouth'] + params['advertising_adopt']) * previous_state['potential_adopters']
    
    
    
    # Output
    return {'delta_potential_adopters': - popu_adopt}

In [8]:
def s_willing_popu(params, 
          substep, 
          state_history, 
          previous_state,
          policy_input):

    
    total_population = previous_state['total_population'] + policy_input['delta_population']
    
    # Output
    return ('total_population', max(math.ceil(total_population),0))

In [9]:
def s_popu_adopt(params, 
          substep, 
          state_history, 
          previous_state,
          policy_input):
    
    potential_adopters = previous_state['potential_adopters'] + policy_input['delta_potential_adopters']
    
    
    # Output
    return ('potential_adopters', max(math.ceil(potential_adopters),0))

In [ ]:
def s_new_location(params, 
                    substep, 
                    state_history, 
                    previous_state,sL,s):
    
    y = 'total_population'
    add_2_total_population = 0
    
    if s > 24 and s % 12 == 0:
           while s < 80: 
            add_2_total_population = 70
    
    elif s < 24 and s % 12 != 0:
            while s > 80:
                add_2_total_population = 50
            
    x = s['total_population'] + add_2_total_population
    
        

    return (y,x)
        
        
s_new_location(0,0,0,0,0,36)

In [122]:
partial_state_update_blocks = [
    {
        'label': 'population dynamics', 
        'policies': {
            'willing_popu': p_willing_popu,
            'popu_increase': p_popu_increase,
        },
        'variables': {
            'total_population': s_willing_popu
            
        }
    },
    {
        'label': 'user adoption dynamics', 
        'policies': {
            'popu_adopt': p_popu_adopt,
            'app_downloads': p_app_downloads,
        },
        'variables': {
            'potential_adopters': s_popu_adopt
        
        }
        
        }
]


In [123]:
sim_config = config_sim({
    "N": 1,
    "T": range(120),
    "M": system_params
})

In [124]:
from cadCAD import configs
del configs[:] # Clear any prior configs

In [125]:
experiment = Experiment()
experiment.append_configs(
    initial_state = initial_state,
    partial_state_update_blocks = partial_state_update_blocks,
    sim_configs = sim_config
)
configs[-1].__dict__

{'sim_config': {'N': 1,
  'T': range(0, 120),
  'M': {'fraction_adopt': 0.6,
   'advertising_adopt': 0.04,
   'word_ofmouth': 0.16},
  'subset_id': 0,
  'subset_window': deque([0, None]),
  'simulation_id': 0,
  'run_id': 0},
 'initial_state': {'total_population': 80000, 'potential_adopters': 2400},
 'seeds': {},
 'env_processes': {},
 'exogenous_states': {},
 'partial_state_updates': [{'label': 'population dynamics',
   'policies': {'willing_popu': <function __main__.p_willing_popu(params, substep, state_history, previous_state)>,
    'popu_increase': <function __main__.p_popu_increase(params, substep, state_history, previous_state)>},
   'variables': {'total_population': <function __main__.s_willing_popu(params, substep, state_history, previous_state, policy_input)>}},
  {'label': 'user adoption dynamics',
   'policies': {'popu_adopt': <function __main__.p_popu_adopt(params, substep, state_history, previous_state)>,
    'app_downloads': <function __main__.p_app_downloads(params, subs

In [126]:
exec_context = ExecutionContext()

simulation = Executor(exec_context=exec_context, configs=configs)
raw_result, tensor_field, sessions = simulation.execute()


                  ___________    ____
  ________ __ ___/ / ____/   |  / __ \
 / ___/ __` / __  / /   / /| | / / / /
/ /__/ /_/ / /_/ / /___/ ___ |/ /_/ /
\___/\__,_/\__,_/\____/_/  |_/_____/
by cadCAD

Execution Mode: local_proc
Configuration Count: 1
Dimensions of the first simulation: (Timesteps, Params, Runs, Vars) = (120, 3, 1, 2)
Execution Method: local_simulations
SimIDs   : [0]
SubsetIDs: [0]
Ns       : [0]
ExpIDs   : [0]
Execution Mode: single_threaded


TypeError: unsupported operand type(s) for +: 'function' and 'int'